## Scraping LinkedIn Job Postings with Python and Selenium

https://medium.com/ai-in-plain-english/scraping-linkedin-job-postings-with-python-and-selenium-by-chatgpt-7edd8450ebce

https://til.simonwillison.net/selenium/selenium-python-macos

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
query = 'Data Scientist'
location = 'United States'

In [6]:
pwd

'/Users/keithlowton/Desktop/Ks/Python code/Scraping'

In [22]:
chromedriver_path = "/Users/keithlowton/Desktop/Ks/Python code/Scraping/chromedriver"

In [23]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options, executable_path=chromedriver_path)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
driver.get('https://www.linkedin.com/login')

email_input = driver.find_element(By.ID, 'username')
password_input = driver.find_element(By.ID, 'password')
email_input.send_keys('your_email')
password_input.send_keys('your_password')
password_input.send_keys(Keys.ENTER)

In [ ]:
time.sleep(10)

In [ ]:
for page_num in range(1, 15):
    url = f'https://www.linkedin.com/jobs/search/?keywords={query}&location={location}&start={25 * (page_num - 1)}'
    driver.get(url)
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(20)  # Wait for 20 seconds

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})

# Extract relevant information from each job posting and store it in a list of dictionaries
data = []
for job_posting in job_postings:
    try:
        job_title = job_posting.find('a', class_='job-card-list__title').get_text().strip()
    except AttributeError:
        job_title = None

    try:
        company_name = job_posting.find('a', class_='job-card-container__link').get_text().strip()
    except AttributeError:
        company_name = None

    try:
        location = job_posting.find('li', class_='job-card-container__metadata-item').get_text().strip()
    except AttributeError:
        location = None

    data.append({
        'Job Title': job_title,
        'Company Name': company_name,
        'Location': location
    })

In [ ]:
df = pd.DataFrame(data)